In [5]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from skopt.space import Dimension, Real
from definitions import PROCESSED_DATA_DIR
from src.data.indicators import rsi, stochRSI, ema
from src.models.strategies import RSIStoch200EMA, BuyAndHold
from src.data.data_pipeline import split_timeseries, evaluate_performance, optimize_hyperparameters

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
btcusdt = pd.read_csv(PROCESSED_DATA_DIR / 'BTCUSDT.csv')
btcusdt.index = pd.DatetimeIndex(btcusdt['open time'])

In [9]:
#Evaluation parameters
num_iteration = 500

#Testset parameters
testset_length = 1000 #30-min intervalls
testset_start = None #random start
random_seed = 1

#Hyperparameters ...#ToDo
hyperparameter_space = [
    Real(10 ** -5, 1.25, "log-uniform", name='max_number_open_trades'),
    Real(10 ** -6, 4096, "log-uniform", name='min_period_to_last_low'), 
    Real(10 ** -6, 4096, "log-uniform", name='max_period_to_last_low'), 
    Real(10 ** -6, 4096, "log-uniform", name='min_absolute_slope_rsi'), 
    Real(10 ** -6, 4096, "log-uniform", name='swing_low_margin'),  
]
num_calls = 30

results = []

for i in range(num_iteration):
    
    result_dict = {}

    #Prepare dataset
    [test_set, training_set] = split_timeseries(btcusdt, testset_length, testset_start, i)

    #Optimize Hyperparameters
    tuned_hps = optimize_hyperparameters(training_set, hyperparameter_space, num_calls) 

    #Apply strategy with tuned hyperparameter on test set
    strategy = RSIStoch200EMA(  #ToDo
        tuned_hps['max_number_open_trades'],
        tuned_hps['min_period_to_last_low'],
        tuned_hps['max_period_to_last_low'],
        tuned_hps['min_absolute_slope_rsi'],
        tuned_hps['swing_low_margin']
    )
    result_dict['tuned_hps'] = tuned_hps

    
    result_dict['strategy_result'] = strategy.apply(
        ohlcv_timeseries=test_set,
        asset_name='BTCUSDT'
    )

    #Evaluate test set performance
    result_dict['test_set_performance'] = evaluate_performance(results[i].pnl_history)
    
    results.append(result_dict)

    
#Evaluate iterations
#ToDo

NameError: name 'tuned_hps' is not defined